In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import norm

## Set time from a certain number of years

In [ ]:
years = 15

endDate = dt.datetime.now()

startDate = endDate - dt.timedelta(days=years*365)

## Create a list of tickers

In [ ]:
sp500url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
data_table = pd.read_html(sp500url)

In [ ]:
tickers = data_table[0]['Symbol'].tolist()
tickers

In [ ]:
for i in range(len(tickers)):
    if tickers[i] == 'BF.B':
        tickers[i] = 'BF-B'
    elif tickers[i] == 'BRK.B':
        tickers[i] = 'BRK-B'

## Download daily Adjusted Close prices for the tickers

In [ ]:
adjClose = pd.DataFrame()

for ticker in tickers:
    adjClose[ticker] = yf.download(ticker, startDate, endDate)['Adj Close']

print(adjClose)

In [ ]:
logReturns = np.log(adjClose / adjClose.shift(1))

logReturns = logReturns.dropna()

print(logReturns)

## Create an equal weight portfolio

In [ ]:
portfolio_value = 1000000
weights = np.array([1/len(tickers)]*len(tickers))
print(weights)

In [ ]:
historical_returns = (logReturns * weights).sum(axis=1)

## Find X-Day Historical Returns

In [ ]:
days = 5
historical_x_day_returns = historical_returns.rolling(window=days).sum()

## Create Covariance Matrix and Calculate Portfolio Standard Deviation

In [ ]:
cov_matrix = logReturns.cov() * 252
portfolio_std_dev = np.sqrt(weights.T @ cov_matrix @ weights)

## Calculate VaR at Different Confidence Levels

In [ ]:
confidence_levels = [0.90, 0.95, 0.99]

VaRs = []
for cl in confidence_levels:
    VaR = portfolio_value * (norm.ppf(1 - cl) * portfolio_std_dev * np.sqrt(days / 252) - historical_returns.mean() * days)
    VaRs.append(VaR)

## Print VaR Results

In [ ]:
print(f'{"Confidence Level":<20} {"Value at Risk":<20}')
print('-' * 40)

for cl, VaR in zip(confidence_levels, VaRs):
    print(f'{cl * 100:>6.0f}%: {"":<8} ${VaR:>10,.2f}')

## Plot the Distribution of Portfolio Returns and Parametric VaR Estimates

In [ ]:
# Convert returns to dollar values for the histogram
historical_x_day_returns_dollar = historical_x_day_returns * portfolio_value

# Plot the histogram
plt.hist(historical_x_day_returns_dollar, bins=50, density=True, alpha=0.5, label=f'{days}-Day Returns')

# Add vertical lines representing VaR at each confidence level
for cl, VaR in zip(confidence_levels, VaRs):
    plt.axvline(x=-VaR, linestyle='--', color='r', label='VaR at {}% Confidence'.format(int(cl * 100)))

plt.xlabel(f'{days}-Day Portfolio Return ($)')
plt.ylabel('Frequency')
plt.title(f'Distribution of Portfolio {days}-Day Returns and Parametric VaR Estimates')
plt.legend()
plt.show()